In [ ]:
# Just to know last time this was run:
import time
print(time.ctime())

# J Using PyMySQL to access MySQL databases

This package contains a pure-Python MySQL client library. In this sense, it does not need to have access to mysql header or library, which is the case for the mysqldb package.
The goal of PyMySQL is to be a drop-in replacement for MySQLdb and work on CPython, PyPy, IronPython and Jython.

It is installed with "pip install pymysql"

We first import the usual libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

This is the import of the library used to connect to MySQl database

In [ ]:
import pymysql

First you need to connect to a database. In our example, we will use the 3MdB database, which needs a password. https://sites.google.com/site/mexicanmillionmodels/

## TIP: DIRECTLY GO TO THE LAST SECTION (USING PANDAS)

### Connect to the database

In [ ]:
user_password = '***' # ask me for the password :-)

In [ ]:
# We create a connector to the database
connector = pymysql.connect(host='3mdb.astro.unam.mx', port=3306, user='OVN_user', passwd=user_password, db='3MdB')

### Use a cursor to send query and receive results

In [ ]:
# The cursor is used to send and receive the quesries to the databse
cur = connector.cursor()

In [ ]:
# Send the query to be executed. It returns the number of lines of the result
cur.execute('select * from `lines` limit 15')

In [ ]:
# get a description of the columns of the query results
cur.description

In [ ]:
# fech all the resulting data into a variable
lines = cur.fetchall()

In [ ]:
# close the cursor once used
cur.close()

In [ ]:
# the result is in a form of tuple of tuples
print(lines)

In [ ]:
# Each element of the first level tuple is a tuple corresponding to a row of the query results
print(len(lines))
print(lines[0])

### Using a cursor that returns a dictionary

In [ ]:
cur_dic = connector.cursor(pymysql.cursors.DictCursor)

In [ ]:
cur_dic.execute('select * from `lines` limit 15')

In [ ]:
lines_dic = cur_dic.fetchall()

In [ ]:
print(lines_dic)

In [ ]:
# Each element of the table is a dictionary corresponding to a row od the query results
print(lines_dic[0])

In [ ]:
# One can easily create a new dictionary than hold the data in columns, better for plotting.
new_dic = {k:np.array([d[k] for d in lines_dic]) for k in lines_dic[0].keys()}

In [ ]:
# The names of the columns are the names use in the database
new_dic['lambda']

In [ ]:
# One can also transform the results into a numpy recarray.
# First step: create a table from the dictionnary
lines_tab = [list(e.values()) for e in lines_dic]
lines_tab

In [ ]:
# Second step: transform the table into a numpy recarray, using the names from the dictionnary
names = list(lines_dic[0].keys())
res = np.rec.fromrecords(lines_tab, names = names)

In [ ]:
res

In [ ]:
res['lambda']

### Example of plotting the result of a query

In [ ]:
# Send the query
N = cur_dic.execute('select O__3__5007A, N__2__6584A, H__1__6563A, oxygen from tab where ref = "HII_CHIm"')

In [ ]:
print(N)

In [ ]:
# obtain the results as a dictionnary
res = cur_dic.fetchall()

In [ ]:
# transform the dictionary into a recarray
data = np.rec.fromrecords([list(e.values()) for e in res], names = list(res[0].keys()))

In [ ]:
# check the data
data[0]

In [ ]:
data['O__3__5007A']

In [ ]:
# Plot the results, using a column as color code
fig, ax = plt.subplots(figsize=(10,7))
scat = ax.scatter(np.log10(data['N__2__6584A'] / data['H__1__6563A']), np.log10(data['O__3__5007A'] / data['H__1__6563A']), 
            c=data['oxygen'], edgecolor='none')
fig.colorbar(scat);

In [ ]:
# Disconnect cursor and connector
cur_dic.close()
connector.close()

### Using pandas library

In [ ]:
import pandas as pd 
import pymysql 
import matplotlib.pyplot as plt

user_password = '***' # ask me for the password :-)
co = pymysql.connect(host='3mdb.astro.unam.mx', db='3MdB', user='OVN_user', passwd=user_password) # change for the right  passwd, just ask me for them!!! 

In [ ]:
res = pd.read_sql("""
SELECT log10(N__2__6584A/H__1__6563A) as n2, 
    log10(O__3__5007A/H__1__4861A) as o3, 
    OXYGEN as O 
FROM tab 
WHERE ref = 'DIG_HR'""", 
                  con=co) 
co.close() 

In [ ]:
print(len(res))

In [ ]:
plt.scatter(res['n2'], res['o3'], c=res['O'], edgecolor='None')
plt.colorbar()

In [ ]:
res

### More on databases, astronomy, SQL and python:

* Astroquery: part of astropy : https://astroquery.readthedocs.io/en/latest/index.html
 * part of astroquery: TAP/TAP+: https://astroquery.readthedocs.io/en/latest/utils/tap.html
 * Gaia TAP+: https://astroquery.readthedocs.io/en/latest/gaia/gaia.html
 * Atomic line list: https://astroquery.readthedocs.io/en/latest/atomic/atomic.html
 * Simbad: https://astroquery.readthedocs.io/en/latest/simbad/simbad.html
 * Vizier: https://astroquery.readthedocs.io/en/latest/vizier/vizier.html
 * This lecture explains step by step how to extract information from the Gaia database: https://allendowney.github.io/AstronomicalData/


* Virtual Observatory : 
 * Cone search: http://voservices.net/spectrum/search_form_cone.aspx
 * SQL interface: http://voservices.net/spectrum/search_form_sql.aspx


* SciServer (needs an account): 
 * https://apps.sciserver.org/dashboard/
 * Introduction to CasJobs: https://skyserver.sdss.org/CasJobs/Guide.aspx
 * Example of Skyquery: http://www.voservices.net/skyquery/Assets/Query/Examples/00_index.aspx
 * Using Python : https://github.com/sciserver/SciScript-Python
 * Example using ython: https://github.com/sciserver/SciScript-Python/blob/master/Examples/Examples_SciScript-Python.ipynb


* An enhanced command line SQL interpreter client for astronomical surveys: https://github.com/mgckind/easyaccess